In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from tqdm import tqdm

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 클래스 정의
class GenderStyleDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        print(f"Loaded {len(self.image_files)} images from {folder_path}")
        self.transform = transform
        self.label_encoder = LabelEncoder()
        self.labels = [self.get_label_from_filename(f) for f in self.image_files]
        self.label_encoder.fit(self.labels)
        self.encoded_labels = self.label_encoder.transform(self.labels)

    def get_label_from_filename(self, filename):
        parts = filename.split('_')
        gender = parts[-1].replace('.jpg', '')
        style = parts[-2]
        return f"{gender}_{style}"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.folder_path, image_file)
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.encoded_labels[idx]
        return image, label

# 경로 설정
train_output_folder = '/content/drive/MyDrive/dataset/crop'
val_output_folder = '/content/drive/MyDrive/dataset/crop_test'

# 학습 데이터셋 및 DataLoader 정의
train_dataset = GenderStyleDataset(train_output_folder, transform=transform)
val_dataset = GenderStyleDataset(val_output_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, drop_last=True)

# 클래스 수 확인
num_classes = len(train_dataset.label_encoder.classes_)
print(f"Number of classes: {num_classes}")
print("Class labels:", train_dataset.label_encoder.classes_)

# ResNet-18 모델 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = resnet18(weights=None)
model.fc = nn.Sequential(
    nn.Dropout(0.5),  # 드롭아웃 추가
    nn.Linear(model.fc.in_features, num_classes)
)
model = model.to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)  # L2 정규화 적용
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # 학습률 스케줄러 추가

Loaded 4070 images from /content/drive/MyDrive/dataset/crop
Loaded 951 images from /content/drive/MyDrive/dataset/crop_test
Number of classes: 31
Class labels: ['M_bold' 'M_hiphop' 'M_hippie' 'M_ivy' 'M_metrosexual' 'M_mods'
 'M_normcore' 'M_sportivecasual' 'W_athleisure' 'W_bodyconscious'
 'W_cityglam' 'W_classic' 'W_disco' 'W_ecology' 'W_feminine'
 'W_genderless' 'W_grunge' 'W_hiphop' 'W_hippie' 'W_kitsch' 'W_lingerie'
 'W_lounge' 'W_military' 'W_minimal' 'W_normcore' 'W_oriental' 'W_popart'
 'W_powersuit' 'W_punk' 'W_space' 'W_sportivecasual']


In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from tqdm import tqdm

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 클래스 정의
class GenderStyleDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        print(f"Loaded {len(self.image_files)} images from {folder_path}")
        self.transform = transform
        self.label_encoder = LabelEncoder()
        self.labels = [self.get_label_from_filename(f) for f in self.image_files]
        self.label_encoder.fit(self.labels)
        self.encoded_labels = self.label_encoder.transform(self.labels)

    def get_label_from_filename(self, filename):
        parts = filename.split('_')
        gender = parts[-1].replace('.jpg', '')
        style = parts[-2]
        return f"{gender}_{style}"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx]
        image_path = os.path.join(self.folder_path, image_file)
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.encoded_labels[idx]
        return image, label

# 경로 설정
train_output_folder = '/content/drive/MyDrive/dataset/crop'
val_output_folder = '/content/drive/MyDrive/dataset/crop_test'

# 학습 데이터셋 및 DataLoader 정의
train_dataset = GenderStyleDataset(train_output_folder, transform=transform)
val_dataset = GenderStyleDataset(val_output_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, drop_last=True)

# 클래스 수 확인
num_classes = len(train_dataset.label_encoder.classes_)
print(f"Number of classes: {num_classes}")
print("Class labels:", train_dataset.label_encoder.classes_)

# ResNet-18 모델 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = resnet18(weights=None)
model.fc = nn.Sequential(
    nn.Dropout(0.5),  # 드롭아웃 추가
    nn.Linear(model.fc.in_features, num_classes)
)
model = model.to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)  # L2 정규화 적용
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # 학습률 스케줄러 추가

# 학습 루프
num_epochs = 100
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    train_accuracy = correct_predictions / total_predictions * 100

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_loader, desc="Validation"):
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            v_loss = criterion(val_outputs, val_labels)
            val_loss += v_loss.item()

            _, val_predicted = torch.max(val_outputs, 1)
            correct_val_predictions += (val_predicted == val_labels).sum().item()
            total_val_predictions += val_labels.size(0)

    val_accuracy = correct_val_predictions / total_val_predictions * 100

    # 학습률 스케줄러 업데이트
    scheduler.step()

    # 에포크별 손실 및 정확도 출력
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Validation accuracy가 가장 높은 모델 저장
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), '/content/drive/MyDrive/checkpoint_new/best_model_last.pth')
        print("New best model saved with Val Accuracy: {:.2f}%".format(best_val_accuracy))

print("모델 학습이 완료되었습니다.")

Loaded 4070 images from /content/drive/MyDrive/dataset/crop
Loaded 951 images from /content/drive/MyDrive/dataset/crop_test
Number of classes: 31
Class labels: ['M_bold' 'M_hiphop' 'M_hippie' 'M_ivy' 'M_metrosexual' 'M_mods'
 'M_normcore' 'M_sportivecasual' 'W_athleisure' 'W_bodyconscious'
 'W_cityglam' 'W_classic' 'W_disco' 'W_ecology' 'W_feminine'
 'W_genderless' 'W_grunge' 'W_hiphop' 'W_hippie' 'W_kitsch' 'W_lingerie'
 'W_lounge' 'W_military' 'W_minimal' 'W_normcore' 'W_oriental' 'W_popart'
 'W_powersuit' 'W_punk' 'W_space' 'W_sportivecasual']


Validation: 100%|██████████| 7/7 [01:30<00:00, 12.86s/it]


Epoch [1/100], Loss: 3.4172, Train Accuracy: 7.21%, Val Loss: 3.2361, Val Accuracy: 6.58%
New best model saved with Val Accuracy: 6.58%


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.62s/it]


Epoch [2/100], Loss: 3.2141, Train Accuracy: 9.58%, Val Loss: 3.1172, Val Accuracy: 10.60%
New best model saved with Val Accuracy: 10.60%


...

Epoch [98/100], Loss: 2.0611, Train Accuracy: 39.29%, Val Loss: 2.4211, Val Accuracy: 30.13%


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.59s/it]


Epoch [99/100], Loss: 2.0811, Train Accuracy: 39.04%, Val Loss: 2.3734, Val Accuracy: 32.37%


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.61s/it]

Epoch [100/100], Loss: 2.0588, Train Accuracy: 40.15%, Val Loss: 2.3884, Val Accuracy: 32.14%
모델 학습이 완료되었습니다.


In [ ]:
# 기존 checkpoint 불러오기
checkpoint_path = '/content/drive/MyDrive/checkpoint_new/best_model_last.pth'

# 추가 학습 루프
num_epochs = 100  # 추가 학습할 에포크 수 지정
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    train_accuracy = correct_predictions / total_predictions * 100

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_loader, desc="Validation"):
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            v_loss = criterion(val_outputs, val_labels)
            val_loss += v_loss.item()

            _, val_predicted = torch.max(val_outputs, 1)
            correct_val_predictions += (val_predicted == val_labels).sum().item()
            total_val_predictions += val_labels.size(0)

    val_accuracy = correct_val_predictions / total_val_predictions * 100

    # 학습률 스케줄러 업데이트
    scheduler.step()

    # 에포크별 손실 및 정확도 출력
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Validation accuracy가 가장 높은 모델 저장
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), '/content/drive/MyDrive/checkpoint_new/best_model_last_2.pth')
        print("New best model saved with Val Accuracy: {:.2f}%".format(best_val_accuracy))

print("추가 모델 학습이 완료되었습니다.")

<ipython-input-4-372cfccfe176>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Checkpoint loaded successfully.


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.69s/it]


Epoch [1/100], Loss: 2.4358, Train Accuracy: 26.92%, Val Loss: 2.6828, Val Accuracy: 24.55%
New best model saved with Val Accuracy: 24.55%


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.69s/it]


...

Epoch [98/100], Loss: 0.8774, Train Accuracy: 79.46%, Val Loss: 1.8315, Val Accuracy: 54.80%


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.60s/it]


Epoch [99/100], Loss: 0.8782, Train Accuracy: 79.16%, Val Loss: 1.8728, Val Accuracy: 53.91%


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.65s/it]

Epoch [100/100], Loss: 0.8769, Train Accuracy: 79.28%, Val Loss: 1.8392, Val Accuracy: 54.24%
추가 모델 학습이 완료되었습니다.


In [ ]:
# 기존 checkpoint 불러오기
checkpoint_path = '/content/drive/MyDrive/checkpoint_new/best_model_last_2.pth'

# 추가 학습 루프
num_epochs = 50  # 추가 학습할 에포크 수 지정
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    train_accuracy = correct_predictions / total_predictions * 100

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_loader, desc="Validation"):
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            v_loss = criterion(val_outputs, val_labels)
            val_loss += v_loss.item()

            _, val_predicted = torch.max(val_outputs, 1)
            correct_val_predictions += (val_predicted == val_labels).sum().item()
            total_val_predictions += val_labels.size(0)

    val_accuracy = correct_val_predictions / total_val_predictions * 100

    # 학습률 스케줄러 업데이트
    scheduler.step()

    # 에포크별 손실 및 정확도 출력
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Validation accuracy가 가장 높은 모델 저장
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), '/content/drive/MyDrive/checkpoint_new/best_model_last_3.pth')
        print("New best model saved with Val Accuracy: {:.2f}%".format(best_val_accuracy))

print("추가 모델 학습이 완료되었습니다.")

<ipython-input-4-689e1a6a4d5c>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Checkpoint loaded successfully.


Validation: 100%|██████████| 7/7 [01:41<00:00, 14.51s/it]


Epoch [1/50], Loss: 1.3551, Train Accuracy: 60.64%, Val Loss: 2.4134, Val Accuracy: 37.05%
New best model saved with Val Accuracy: 37.05%


Validation: 100%|██████████| 7/7 [00:17<00:00,  2.47s/it]


...

Epoch [48/50], Loss: 0.2908, Train Accuracy: 95.99%, Val Loss: 1.6189, Val Accuracy: 61.94%


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.66s/it]


Epoch [49/50], Loss: 0.2929, Train Accuracy: 95.59%, Val Loss: 1.6921, Val Accuracy: 60.71%


Validation: 100%|██████████| 7/7 [00:19<00:00,  2.83s/it]

Epoch [50/50], Loss: 0.2828, Train Accuracy: 95.79%, Val Loss: 1.6577, Val Accuracy: 61.16%
추가 모델 학습이 완료되었습니다.


In [ ]:
# 기존 checkpoint 불러오기
checkpoint_path = '/content/drive/MyDrive/checkpoint_new/best_model_last_3.pth'

# 추가 학습 루프
num_epochs = 100  # 추가 학습할 에포크 수 지정
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    train_accuracy = correct_predictions / total_predictions * 100

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_predictions = 0
    with torch.no_grad():
        for val_images, val_labels in tqdm(val_loader, desc="Validation"):
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            v_loss = criterion(val_outputs, val_labels)
            val_loss += v_loss.item()

            _, val_predicted = torch.max(val_outputs, 1)
            correct_val_predictions += (val_predicted == val_labels).sum().item()
            total_val_predictions += val_labels.size(0)

    val_accuracy = correct_val_predictions / total_val_predictions * 100

    # 학습률 스케줄러 업데이트
    scheduler.step()

    # 에포크별 손실 및 정확도 출력
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Validation accuracy가 가장 높은 모델 저장
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), '/content/drive/MyDrive/checkpoint_new/best_model_last_4.pth')
        print("New best model saved with Val Accuracy: {:.2f}%".format(best_val_accuracy))

print("추가 모델 학습이 완료되었습니다.")

<ipython-input-2-f222969c7ae0>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Checkpoint loaded successfully.


Validation: 100%|██████████| 7/7 [01:25<00:00, 12.24s/it]


Epoch [1/100], Loss: 0.7549, Train Accuracy: 78.38%, Val Loss: 2.3384, Val Accuracy: 46.32%
New best model saved with Val Accuracy: 46.32%


Validation: 100%|██████████| 7/7 [00:17<00:00,  2.57s/it]


...

Epoch [98/100], Loss: 0.0908, Train Accuracy: 99.04%, Val Loss: 1.7605, Val Accuracy: 62.39%


Validation: 100%|██████████| 7/7 [00:17<00:00,  2.56s/it]


Epoch [99/100], Loss: 0.0956, Train Accuracy: 98.74%, Val Loss: 1.6940, Val Accuracy: 62.28%


Validation: 100%|██████████| 7/7 [00:18<00:00,  2.69s/it]

Epoch [100/100], Loss: 0.0870, Train Accuracy: 99.09%, Val Loss: 1.7125, Val Accuracy: 63.28%
추가 모델 학습이 완료되었습니다.
